In [2]:
!pip install pyarrow

     |████████████████████████████████| 41.0 MB 56 kB/s  eta 0:00:01     |█████████████████████████▏      | 32.3 MB 2.2 MB/s eta 0:00:04


In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

--2024-05-23 23:22:24--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.239.192.206, 54.239.192.100, 54.239.192.95, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.239.192.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet’

yellow_tripdata_202 100%[===================>]  45.46M  1.30MB/s    in 86s     

2024-05-23 23:23:53 (541 KB/s) - ‘yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]



In [7]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

--2024-05-23 23:27:30--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.239.192.100, 54.239.192.206, 54.239.192.95, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.239.192.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47748012 (46M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-02.parquet.1’

yellow_tripdata_202 100%[===================>]  45.54M  2.70MB/s    in 3m 10s  

2024-05-23 23:30:42 (246 KB/s) - ‘yellow_tripdata_2023-02.parquet.1’ saved [47748012/47748012]



In [1]:
import pandas as pd

df1=pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

df2=pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

In [9]:
len(df1.columns)

19

In [2]:
df1.tpep_dropoff_datetime = pd.to_datetime(df1.tpep_dropoff_datetime)
df1.tpep_pickup_datetime = pd.to_datetime(df1.tpep_pickup_datetime)

df1['duration'] = df1.tpep_dropoff_datetime - df1.tpep_pickup_datetime
df1['duration'] = df1.duration.apply(lambda td: td.total_seconds() / 60)


df2.tpep_dropoff_datetime = pd.to_datetime(df2.tpep_dropoff_datetime)
df2.tpep_pickup_datetime = pd.to_datetime(df2.tpep_pickup_datetime)

df2['duration'] = df2.tpep_dropoff_datetime - df2.tpep_pickup_datetime
df2['duration'] = df2.duration.apply(lambda td: td.total_seconds() / 60)


df1.duration.std() #df1.duration.describe()

42.59435124195458

In [ ]:
df1[(df1['duration'] >=1)  & (df1['duration'] <= 60)].shape[0] / df1.shape[0]

0.9812202822125979

In [3]:
from sklearn.feature_extraction import DictVectorizer

categorical = ['PULocationID', 'DOLocationID']

df1[categorical] = df1[categorical].astype(str)

train_dicts = df1[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

X_train

<3066766x518 sparse matrix of type '<class 'numpy.float64'>'
	with 6133532 stored elements in Compressed Sparse Row format>

In [4]:
df2[categorical] = df2[categorical].astype(str)

val_dicts = df2[categorical].to_dict(orient='records')


X_val = dv.transform(val_dicts)

In [5]:
target = 'duration'
y_train = df1[target].values

y_val = df2[target].values

In [6]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [7]:
from sklearn.metrics import mean_squared_error

y_pred = lr.predict(X_train)

y_val = lr.predict(X_val)

In [8]:
mean_squared_error(y_train, y_pred, squared=False)

41.99649697672271

In [9]:
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

0.0